In [1]:
import os
import torch

import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import time
import copy
import numpy as np
import random
import pydicom
import nibabel as nib
import matplotlib.pyplot as plt
import pandas as pd
import datetime

In [2]:
from torch.optim import lr_scheduler
from torchvision import datasets
from efficientnet_pytorch import EfficientNet
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score
from sklearn.metrics import classification_report, roc_curve
import logging

from pathlib import Path
from PIL import Image
import SimpleITK as sitk
from numpy import asarray
from math import sqrt
from scipy.special import ndtri
from numpy import argmax

In [3]:
class Medical_dataset(torch.utils.data.Dataset):

    def __init__(self, first_layer_files, second_layer_files, third_layer_files, labels, transforms):

        self.first_layer_files = first_layer_files
        self.second_layer_files = second_layer_files
        self.third_layer_files = third_layer_files
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        return min(len(self.first_layer_files), len(self.second_layer_files), len(self.third_layer_files))
    
    def __getitem__(self, idx):

        # first layer will be dcm files
        img_path_dcm = self.first_layer_files[idx]
        ds_A = pydicom.read_file(img_path_dcm)
        img_A = np.array(ds_A.pixel_array, dtype=np.float32)

        min, max = np.min(img_A), np.max(img_A)
        img_A = 255 - ((img_A - min) / (max - min)) * 255
        PIL_image_A = Image.fromarray(img_A.astype(np.uint8))

        # second layer will be nii files
        img_path = self.second_layer_files[idx]
        ds_B = nib.load(img_path)
        img_B = np.array(ds_B.get_fdata(), dtype=np.float32)

        if len(img_B.shape) == 4:
            img_B = img_B[:,:,0,0]
        else:
            min, max = np.min(img_B), np.max(img_B)
            img_B = ((img_B - min) / (max - min)) * 255
            PIL_image_B = Image.fromarray(np.transpose(img_B).astype(np.uint8))

        # third layer will be nii files
        img_path = self.third_layer_files[idx]
        ds_C = nib.load(img_path)
        img_C = np.array(ds_C.get_fdata(), dtype=np.float32)

        if len(img_C.shape) == 4:
            img_C = img_C[:,:,0,0]
        else:
            min, max = np.min(img_C), np.max(img_C)
            img_C = ((img_C - min) / (max - min)) * 255
            PIL_image_C = Image.fromarray(np.transpose(img_C).astype(np.uint8))

        return self.transforms(PIL_image_A), self.transforms(PIL_image_B), self.transforms(PIL_image_C), self.labels[idx]

In [4]:
resol = 1024

train_transforms = transforms.Compose([
    transforms.Resize((resol, resol)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

valid_transforms = transforms.Compose([
    transforms.Resize((resol, resol)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

In [5]:
# make dataset for train and valid based on class above
first_layer_files_list = []
fl_dir = '/data4/vindr-cxr/train/'
files = sorted(os.listdir(fl_dir))
for file in files:
    first_layer_files_list.append(fl_dir + file)

second_layer_files_list = []
sl_dir = '/data4/20231130_tisepx_vinbig/train_lung/'
files = sorted(os.listdir(sl_dir))
for file in files:
    second_layer_files_list.append(sl_dir + file)

third_layer_files_list = []
tl_dir = '/data4/20231130_tisepx_vinbig/train_pulmonaryLung/'
files = sorted(os.listdir(tl_dir))
for file in files:
    third_layer_files_list.append(tl_dir + file)

img_list = sorted(os.listdir(fl_dir))
id_list = []
for img in img_list:
    id_list.append(img.split('.')[0])

labels = []
metadata_dir = '/data4/vindr-cxr/train.csv'
df = pd.read_csv(metadata_dir)

for i in id_list:
    labels.append(df.loc[df['image_id'] == i, 'class_id'].iloc[0])

# count numver of values in each class
from collections import Counter
print(Counter(labels))
print(len(labels))

Counter({14: 10606, 0: 1127, 3: 878, 13: 522, 11: 496, 7: 251, 10: 241, 9: 224, 8: 221, 6: 134, 5: 111, 2: 91, 4: 46, 1: 30, 12: 22})
15000


In [6]:
# split train and valid
length = len(first_layer_files_list)
np.random.seed(555)
indices = np.arange(length)
np.random.shuffle(indices)

test_split = int(np.floor(0.2 * length))
val_split = int(np.floor(0.1 * length))
test_indices, val_indices, train_indices = indices[:test_split], indices[test_split:test_split+val_split], indices[test_split+val_split:]

train_x1 = [first_layer_files_list[i] for i in train_indices]
train_x2 = [second_layer_files_list[i] for i in train_indices]
train_x3 = [third_layer_files_list[i] for i in train_indices]
train_y = [labels[i] for i in train_indices]

val_x1 = [first_layer_files_list[i] for i in val_indices]
val_x2 = [second_layer_files_list[i] for i in val_indices]
val_x3 = [third_layer_files_list[i] for i in val_indices]
val_y = [labels[i] for i in val_indices]

test_x1 = [first_layer_files_list[i] for i in test_indices]
test_x2 = [second_layer_files_list[i] for i in test_indices]
test_x3 = [third_layer_files_list[i] for i in test_indices]
test_y = [labels[i] for i in test_indices]

print(f'num of train dataset: {len(train_x1)}, valid dataset: {len(val_x2)}, test dataset: {len(test_x3)}')

num of train dataset: 10500, valid dataset: 1500, test dataset: 3000


In [7]:
# Setting up training enviornment
from efficientnet_pytorch import EfficientNet
model_name = 'efficientnet-b5'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EfficientNet.from_pretrained(model_name,in_channels=3, num_classes=15)
# use multiple gpu
model = nn.DataParallel(model)
model.to(device)

loss_function = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), 
                         lr = 0.05,
                         momentum=0.9,
                         weight_decay=1e-4)
lmbda = lambda epoch: 0.98739
scheduler = optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lmbda)

batch_size  = 32
random_seed = 555
random.seed(random_seed)
torch.manual_seed(random_seed)

Loaded pretrained weights for efficientnet-b5


In [8]:
# train and valid dataset
train_dataset = Medical_dataset(train_x1, train_x2, train_x3, train_y, train_transforms)
valid_dataset = Medical_dataset(val_x1, val_x2, val_x3, val_y, valid_transforms)
test_dataset = Medical_dataset(test_x1, test_x2, test_x3, test_y, valid_transforms)

# dataloader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

In [29]:
# train and valid for 3 channels EfficientNet
best_metric = 100000

epoch_loss_values = []
metric_values = []
auroc_values = []
acc_values = []

for epoch in range(1, 100):
    # train
    start_time = time.time()
    epoch_loss = 0
    epoch_samples = 0
    step = 0
    model.train()
    for batch_data in train_loader:
        step += 1
        # unsquezee each channel
        x1, x2, x3, y = batch_data[0].cpu(), batch_data[1].cpu(), batch_data[2].cpu(), batch_data[3].cuda()
        inputs = torch.cat([x1, x2, x3], axis=1)

        # make y to one hot encoding
        y = F.one_hot(y, num_classes=15).cuda().float()

        optimizer.zero_grad()
        y_hat = model(inputs)
        
        loss = loss_function(y_hat, y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item() * y.size(0)
        epoch_samples += y.size(0)
    scheduler.step()
    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    print(f'epoch {epoch} loss is {epoch_loss:.4f}')

    # make checkpoint
    if epoch % 10 == 0:
        torch.save(model.state_dict(), '/data4/vin_model_weights/' + '_epoch_' + str(epoch) + '_checkpoint.pth')
        print('saved model for checkpoint')

    # validation
    model.eval()
    val_loss_values = []
    val_loss = 0
    with torch.no_grad():
        y_pred = torch.tensor([], dtype=torch.float32, device=device)
        y_labels = torch.tensor([], dtype=torch.float32, device=device)

        for val_data in valid_loader:
            x1, x2, x3, y = val_data[0].cpu(), val_data[1].cpu(), val_data[2].cpu(), val_data[3].cuda()
            val_images = torch.cat((x1, x2, x3), 1).float()

            val_outputs = model(val_images)
            val_labels = F.one_hot(y, num_classes=15).cuda().float()
            
            val_loss = loss_function(val_outputs, val_labels)

            y_pred = torch.cat([y_pred, torch.sigmoid(val_outputs)], dim=0)
            y_labels = torch.cat([y_labels, val_labels], dim=0)

            val_loss_values.append(val_loss.item())

        val_loss = np.sum(val_loss_values) / len(val_loss_values)
        y_pred, y_labels = y_pred.cpu(), y_labels.cpu()
        result = roc_auc_score(y_labels, y_pred)
        acc_metric = accuracy_score(y_labels, (y_pred>0.5).float())

        val_result = np.mean(val_loss_values)
        metric_values.append(val_result)
        
        auroc_values.append(result)
        acc_values.append(acc_metric)
        
        if val_result < best_metric:
            best_metric = val_result
            torch.save(model.state_dict(), '/data4/vin_model_weights/' + '_epoch_' + str(epoch) + '_best_metric_model.pth')
            print('saved new best model')

        print(f'epoch {epoch} val loss is {val_result:.4f}')
        print(f'epoch {epoch} val auroc is {result:.4f}')
        print(f'epoch {epoch} val acc is {acc_metric:.4f}')

    print(f'epoch {epoch} took {(time.time()-start_time)/60:.2f} min')

epoch 1 loss is 2.8673
saved new best model
epoch 1 val loss is 0.0906
epoch 1 val auroc is 0.8493
epoch 1 val acc is 0.6440
epoch 1 took 117.28 min


/usr/local/lib/python3.8/dist-packages/pydicom/pixel_data_handlers/pillow_handler.py:238: UserWarning: The (0028,0101) 'Bits Stored' value (12-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pydicom/pixel_data_handlers/pillow_handler.py:238: UserWarning: The (0028,0101) 'Bits Stored' value (14-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  warnings.warn(


epoch 2 loss is 2.7088
saved new best model
epoch 2 val loss is 0.0870
epoch 2 val auroc is 0.8701
epoch 2 val acc is 0.6553
epoch 2 took 117.09 min


/usr/local/lib/python3.8/dist-packages/pydicom/pixel_data_handlers/pillow_handler.py:238: UserWarning: The (0028,0101) 'Bits Stored' value (12-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pydicom/pixel_data_handlers/pillow_handler.py:238: UserWarning: The (0028,0101) 'Bits Stored' value (14-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  warnings.warn(


epoch 3 loss is 2.5585
epoch 3 val loss is 0.0969
epoch 3 val auroc is 0.8747
epoch 3 val acc is 0.5800
epoch 3 took 117.82 min


...

/usr/local/lib/python3.8/dist-packages/pydicom/pixel_data_handlers/pillow_handler.py:238: UserWarning: The (0028,0101) 'Bits Stored' value (12-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pydicom/pixel_data_handlers/pillow_handler.py:238: UserWarning: The (0028,0101) 'Bits Stored' value (14-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  warnings.warn(


epoch 93 loss is 0.2923
epoch 93 val loss is 0.1165
epoch 93 val auroc is 0.8307
epoch 93 val acc is 0.6607
epoch 93 took 116.41 min


/usr/local/lib/python3.8/dist-packages/pydicom/pixel_data_handlers/pillow_handler.py:238: UserWarning: The (0028,0101) 'Bits Stored' value (14-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pydicom/pixel_data_handlers/pillow_handler.py:238: UserWarning: The (0028,0101) 'Bits Stored' value (12-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  warnings.warn(


epoch 94 loss is 0.2633
epoch 94 val loss is 0.1284
epoch 94 val auroc is 0.8424
epoch 94 val acc is 0.6573
epoch 94 took 116.28 min


/usr/local/lib/python3.8/dist-packages/pydicom/pixel_data_handlers/pillow_handler.py:238: UserWarning: The (0028,0101) 'Bits Stored' value (14-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pydicom/pixel_data_handlers/pillow_handler.py:238: UserWarning: The (0028,0101) 'Bits Stored' value (12-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  warnings.warn(


In [12]:
metric_values = []
auroc_values = []
acc_values = []

model = EfficientNet.from_pretrained(model_name,in_channels=3, num_classes=15)
model = nn.DataParallel(model)
model.to(device)

model.load_state_dict(torch.load('/data4/vin_model_weights/_epoch_5_best_metric_model.pth'))
model.eval()

test_loss_values = []
test_loss = 0

with torch.no_grad():

    y_pred = torch.tensor([], dtype=torch.float32, device=device)
    y_labels = torch.tensor([], dtype=torch.float32, device=device)

    for test_data in test_loader:

        x1, x2, x3, y = test_data[0].cpu(), test_data[1].cpu(), test_data[2].cpu(), test_data[3].cuda()
        outputs = torch.cat([x1, x2, x3], axis=1)

        # make y to one hot encoding
        test_outputs = model(outputs)
        test_labels = F.one_hot(y, num_classes=15).cuda().float()

        test_loss = loss_function(test_outputs, test_labels)

        y_pred = torch.cat([y_pred, torch.sigmoid(test_outputs)], dim=0)
        y_labels = torch.cat([y_labels, test_labels], dim=0)

        test_loss_values.append(test_loss.item())

    test_loss = np.sum(test_loss_values) / len(test_loss_values)

    y_pred, y_labels = y_pred.cpu(), y_labels.cpu()
    result = roc_auc_score(y_labels, y_pred)
    acc_metric = accuracy_score(y_labels, (y_pred>0.5).float())

    test_result = np.mean(test_loss_values)
    metric_values.append(test_result)

    auroc_values.append(result)
    acc_values.append(acc_metric)

    print(f'test loss is {test_loss:.4f}')
    print(f'test auroc is {result:.4f}')
    print(f'test acc is {acc_metric:.4f}')

Loaded pretrained weights for efficientnet-b5


/usr/local/lib/python3.8/dist-packages/pydicom/pixel_data_handlers/pillow_handler.py:238: UserWarning: The (0028,0101) 'Bits Stored' value (12-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pydicom/pixel_data_handlers/pillow_handler.py:238: UserWarning: The (0028,0101) 'Bits Stored' value (14-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  warnings.warn(


test loss is 0.0729
test auroc is 0.8853
test acc is 0.6927


In [87]:
# add grad-cam to model
import cv2
from torch.autograd import Function
from torchvision import models
from torchvision import utils
from PIL import Image
import matplotlib.cm as cm

class FeatureExtractor():
    """ Class for extracting activations and 
    registering gradients from targetted intermediate layers """

    def __init__(self, model, target_layers):
        self.model = model
        self.target_layers = target_layers
        self.gradients = []

    def save_gradient(self, grad):
        self.gradients.append(grad)

    def __call__(self, x):

        outputs = []
        self.gradients = []
        for name, module in self.model._modules.items():
            if name == 'fc':
                x = x.view(x.size(0), -1)
            x = module(x)
            if name in self.target_layers:
                x.register_hook(self.save_gradient)
                outputs += [x]
        return outputs, x


class ModelOutputs():
    """ Class for making a forward pass, and getting:
    1. The network output.
    2. Activations from intermeddiate targetted layers.
    3. Gradients from intermeddiate targetted layers. """

    def __init__(self, model, target_layers):
        self.model = model
        self.feature_extractor = FeatureExtractor(self.model, target_layers)

    def get_gradients(self):
        return self.feature_extractor.gradients
    
    def __call__(self, x):

        target_activations, output  = self.feature_extractor(x)
        output = torch.sigmoid(output)
        return target_activations, output
    
def preprocess_image(img):
    means=[0.485, 0.456, 0.406]
    stds=[0.229, 0.224, 0.225]

    preprocessed_img = img.copy()[:, :, ::-1]
    for i in range(3):

        preprocessed_img[:, :, i] = preprocessed_img[:, :, i] - means[i]
        preprocessed_img[:, :, i] = preprocessed_img[:, :, i] / stds[i]

    preprocessed_img = \
        np.ascontiguousarray(np.transpose(preprocessed_img, (2, 0, 1)))
    preprocessed_img = torch.from_numpy(preprocessed_img)

    preprocessed_img.unsqueeze_(0)
    input = preprocessed_img.requires_grad_(True)
    return input

def show_cam_on_image(img, mask, file_name):
    
        heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
        heatmap = np.float32(heatmap) / 255
    
        cam = heatmap + np.float32(img)
        cam = cam / np.max(cam)
    
        cv2.imwrite(file_name, np.uint8(255 * cam))

def get_cam(model, input_image, class_idx, file_name):

    model.eval()
    output = model(input_image.cuda())
    # output shape is (batch_size, num_classes)
    output = output[:, class_idx]

    # one of the differentiated Tensors does not require grad
    # so we have to manually set it to true
    print(type(input_image))
    input_image.requires_grad_(True)
    grads = torch.autograd.grad(output, input_image, allow_unused= True)
    print(grads)



model = EfficientNet.from_pretrained(model_name,in_channels=1, num_classes=15)
model.to(device)

# get grad-cam for each class
for i in range(15):
    get_cam(model, test_dataset[i][0].unsqueeze(0), i, '/data4/vin_model_weights/' + str(i) + '_class_grad_cam.jpg')

Loaded pretrained weights for efficientnet-b5


/usr/local/lib/python3.8/dist-packages/pydicom/pixel_data_handlers/pillow_handler.py:238: UserWarning: The (0028,0101) 'Bits Stored' value (12-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  warnings.warn(


<class 'torch.Tensor'>
(None,)
<class 'torch.Tensor'>
(None,)
<class 'torch.Tensor'>
(None,)
<class 'torch.Tensor'>
(None,)
<class 'torch.Tensor'>
(None,)
<class 'torch.Tensor'>
(None,)


/usr/local/lib/python3.8/dist-packages/pydicom/pixel_data_handlers/pillow_handler.py:238: UserWarning: The (0028,0101) 'Bits Stored' value (14-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  warnings.warn(


<class 'torch.Tensor'>
(None,)
<class 'torch.Tensor'>
(None,)
<class 'torch.Tensor'>
(None,)
<class 'torch.Tensor'>
(None,)
<class 'torch.Tensor'>
(None,)
<class 'torch.Tensor'>
(None,)
<class 'torch.Tensor'>
(None,)
<class 'torch.Tensor'>
(None,)
<class 'torch.Tensor'>
(None,)


In [71]:
print(test_dataset[0])

(tensor([[[-0.9843, -0.9843, -0.9843,  ..., -0.9137, -0.9137, -0.9059],
         [-0.9843, -0.9843, -0.9922,  ..., -0.9137, -0.9137, -0.9137],
         [-0.9843, -0.9922, -0.9922,  ..., -0.9216, -0.9216, -0.9216],
         ...,
         [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000]]]), tensor([[[-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         ...,
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.]]]), tensor([[[-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         ...,
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
        

In [36]:
sample = test_dataset[0][0]
print(sample.shape)
uns_sample = sample.unsqueeze(0)
print(uns_sample.shape)

torch.Size([1, 1024, 1024])
torch.Size([1, 1, 1024, 1024])


In [38]:
sample_feature = model.extract_features(uns_sample.cuda())

In [39]:
print(sample_feature.shape)

torch.Size([1, 2048, 32, 32])


In [47]:
sample_output = model(uns_sample.cuda())

In [49]:
print(sample_output)

tensor([[-0.0151, -0.0895, -0.0134, -0.0242, -0.1249,  0.0150, -0.0504, -0.0077,
         -0.1312,  0.1254, -0.0041,  0.0808, -0.1009, -0.0046,  0.0095]],
       device='cuda:0', grad_fn=<AddmmBackward0>)


In [78]:
print(sample_output.shape)

torch.Size([1, 15])


In [50]:
sig_output = torch.sigmoid(sample_output)
print(sig_output)

tensor([[0.4962, 0.4776, 0.4966, 0.4939, 0.4688, 0.5038, 0.4874, 0.4981, 0.4673,
         0.5313, 0.4990, 0.5202, 0.4748, 0.4988, 0.5024]], device='cuda:0',
       grad_fn=<SigmoidBackward0>)
